## **PONTIFÍCIA UNIVERSIDADE CATÓLICA DE MINAS GERAIS - NÚCLEO DE EDUCAÇÃO A DISTÂNCIA**

### Pós-graduação Lato Sensu em Ciência de Dados e Big Data



# **1. Coleta dos dados**

In [ ]:
!pip install basedosdados  ## Instalação da biblioteca de acesso à base

In [ ]:
import numpy as np
import pandas as pd
import basedosdados as bd

### **1.1. Relação Anual de Informações Sociais (RAIS)**



**Para limitar a quantidade de registros, vamos selecionar apenas o ano de 2019 e as empresas do estado do Ceará**




In [ ]:
sql_query = """
SELECT   * 
FROM `basedosdados.br_me_rais.microdados_estabelecimentos` 
WHERE ano = 2019 AND sigla_uf = 'CE'
"""
bd.download(savepath='BD_Rais_2019_CE.csv', query=sql_query, billing_project_id= 'tcc-rais', index=False)

### **1.2.Municípios do Estado do Ceará**

In [ ]:
sql_query = """
SELECT * FROM `basedosdados.br_bd_diretorios_brasil.municipio` 
WHERE sigla_uf = 'CE'
"""
bd.download(savepath='BD_Municipios_CE.csv', query=sql_query, billing_project_id= 'tcc-rais', index=False)

In [ ]:
df_MunicipiosCE = pd.read_csv('./BD_Municipios_CE.csv')
df_MunicipiosCE.info()

### **1.3. Cadastro Nacional de Atividades Econômicas (CNAE)**

In [ ]:
from pandas.io.json import json_normalize
import json

In [ ]:
# Obtendo dados via API 
df_cnae_subclasses = pd.read_json('https://servicodados.ibge.gov.br/api/v2/cnae/subclasses')

# Normalizando arquivo
cnae_subclasses = df_cnae_subclasses.to_dict(orient='records')
df_CNAE = pd.json_normalize(cnae_subclasses, sep = '_')

# Renomeando as colunas
df_CNAE.rename(columns={'id': 'subclasse_id', 'descricao': 'subclasse_descricao', 'observacoes': 'subclasse_observacoes',
                         'atividades': 'subclasse_atividades', 'classe_grupo_id': 'grupo_id', 'classe_grupo_descricao': 'grupo_descricao',
                         'classe_grupo_divisao_id': 'divisao_id', 'classe_grupo_divisao_descricao': 'divisao_descricao',
                         'classe_grupo_divisao_secao_id': 'secao_id', 'classe_grupo_divisao_secao_descricao': 'secao_descricao'}, inplace=True)
# Ajustando Tipo da Coluna
tipo_coluna = { "subclasse_id": "string"}
df_CNAE = df_CNAE.astype(tipo_coluna)

## colocando um "0" a esquerda  
df_CNAE['subclasse_id'] = df_CNAE['subclasse_id'].str.zfill(7)

df_CNAE.head()

In [ ]:
df_CNAE.to_csv('BD_CNAE2.csv', index=False)

### **1.4. Tabela de Natureza Jurídica**

In [ ]:
# Importação das bibliotecas a serem utilizadas
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
# Acessar página Web
req = requests.get('https://concla.ibge.gov.br/estrutura/natjur-estrutura/natureza-juridica-2018.html')
if req.status_code == 200:
    print('Requisição bem sucedida!')
    content = req.content

In [ ]:
# Utilização da biblioteca BeautifulSoup para obtenção dos dados
soup = BeautifulSoup(content, 'html.parser')
tabela = soup.find_all(href=re.compile("2018"))

In [ ]:
## Extrair as informações da Lista obtida da página HTML e transformar em um dataframe

nat_lista = [linha.text for linha in tabela]   ## Obtendo só o conteúdo em texto do retorno da extração realizada
nat_lista = nat_lista[2:]                      ## Visualizando a lista, vemos que as duas primeiras linhas pode ser descartadas

nat_cod = []
nat_desc = []

for i in range(0,len(nat_lista)):
  nat_lista_partes = nat_lista[i].split(' ',2)
  nat_cod.append(nat_lista_partes[0][0:3]+nat_lista_partes[0][4])
  nat_desc.append(nat_lista_partes[2])

df_Nat_juridica = pd.DataFrame(list(zip(nat_cod,nat_desc)), columns = ['cod_nat_juridica','desc_nat_juridica'])
df_Nat_juridica.head()

In [ ]:
# Salva o dataframe em um arquivo CSV
df_Nat_juridica.to_csv('BD_Nat_Juridica.csv', index=False)